<h1>Introduction</h1>
<p>The purpose of this challenge is to use reservation and visitation data to predict the total number of restaurant visitors for future days.</p> 
<p>The Japanese restaurants are owned by Recruit Holdings.
</p>

<h5>Datasets</h5>
<p>Available datasets come from following sources:</p>
<ul> 
    <li>Hot Pepper Gourmet (a restaurant review service)</li>
    <li>AirREGI (a restaurant point of sales service)</li>
    <li>Restaurant Board (reservation log management software)</li>
    </ul>
    
<p>The datasets contain daily and hourly observations. This makes it a Time Series Forecasting problem.</p>
    

<h5>Business background</h5>
<p>One common predicament is that restaurants need to know how many customers to expect each day to effectively purchase ingredients and schedule staff members. This forecast isn't easy to make because many unpredictable factors affect restaurant attendance, like weather and local competition. It's even harder for newer restaurants with little historical data.
</p>

<h2>Load Data</h2>

In [15]:
import datetime
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import datetime as dt
import calendar

%matplotlib inline

In [17]:
#AIR_RESERVE = 'air_reserve'
#AIR_STORE_INFO = 'air_store_info'
#AIR_VISIT_DATA = 'air_visit_data'
#DATE_INFO = 'date_info'
#HPG_RESERVE = 'hpg_reserve'
#HPG_STORE_INFO = 'hpg_store_info'
#STORE_ID_RELATION = 'store_id_relation'
#SAMPLE_SUBMISSION = 'sample_submission'

#data = {
#    AIR_RESERVE:       pd.read_csv('../input/air_reserve.csv', 
#                                   parse_dates=['visit_datetime', 'reserve_datetime']),
#    AIR_STORE_INFO:    pd.read_csv('../input/air_store_info.csv'),
#    AIR_VISIT_DATA:    pd.read_csv('../input/air_visit_data.csv', 
#                                   parse_dates=['visit_date']),
#    HPG_RESERVE:       pd.read_csv('../input/hpg_reserve.csv', 
#                                   parse_dates=['visit_datetime', 'reserve_datetime']),
#    HPG_STORE_INFO:    pd.read_csv('../input/hpg_store_info.csv'),
#    DATE_INFO:         pd.read_csv('../input/date_info.csv', 
#                                   parse_dates=['calendar_date']),
#    STORE_ID_RELATION: pd.read_csv('../input/store_id_relation.csv'),
#    SAMPLE_SUBMISSION: pd.read_csv('../input/sample_submission.csv')
#}

<h2>Tables STORE ID RELATION and HPG RESERVE</h2>

In [18]:
#data[HPG_RESERVE].head()

In [19]:
#data[STORE_ID_RELATION].head()

In [20]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html
# DataFrame.merge(right, 
#                how='inner', 
#                on=None, 
#                left_on=None, right_on=None, 
#                left_index=False, right_index=False, 
#                sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)

# right : DataFrame or named Series Object to merge with.
# how : {‘left’, ‘right’, ‘outer’, ‘inner’}, default ‘inner’
# inner: use intersection of keys from both frames, similar to a SQL inner join; 
#        preserve the order of the left keys.

#  on : label or list
#      Column or index level names to join on.
#      These must be found in both DataFrames. 
#      If on is None and not merging on indexes then this defaults to the intersection of the 
#      columns in both DataFrames.

#data[HPG_RESERVE] = pd.merge(data[HPG_RESERVE], data[STORE_ID_RELATION],how='inner', on=['hpg_store_id'])
#data[HPG_RESERVE].tail(50)

In [27]:
data = {
    'tra': pd.read_csv('../input/air_visit_data.csv'),
    'as': pd.read_csv('../input/air_store_info.csv'),
    'hs': pd.read_csv('../input/hpg_store_info.csv'),
    'ar': pd.read_csv('../input/air_reserve.csv'),
    'hr': pd.read_csv('../input/hpg_reserve.csv'),
    'id': pd.read_csv('../input/store_id_relation.csv'),
    'tes': pd.read_csv('../input/sample_submission.csv'),
    'hol': pd.read_csv('../input/date_info.csv').rename(columns={'calendar_date':'visit_date'})
    }




In [28]:
#data['hr'] = pd.merge(data['hr'], data['id'], how='inner', on=['hpg_store_id'])

In [29]:
data['hr'].head(50)

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13
5,hpg_28bdf7a336ec6a7b,2016-01-01 17:00:00,2016-01-01 15:00:00,2
6,hpg_2a01a042bca04ad9,2016-01-01 17:00:00,2016-01-01 17:00:00,2
7,hpg_2a84dd9f4c140b82,2016-01-01 17:00:00,2016-01-01 15:00:00,2
8,hpg_2ad179871696901f,2016-01-01 17:00:00,2016-01-01 13:00:00,2
9,hpg_2c1d989eedb0ff83,2016-01-01 17:00:00,2016-01-01 15:00:00,6


In [30]:
len(data['hr']['hpg_store_id'].unique())

13325

In [31]:
data['hr'].describe()

,reserve_visitors
count,2.000320e+06
mean,5.073785e+00
std,5.416172e+00
min,1.000000e+00
25%,2.000000e+00
50%,3.000000e+00
75%,6.000000e+00
max,1.000000e+02
